# Preparação

## Imports

In [1]:
import os
import duckdb
%load_ext sql

from dotenv import load_dotenv

load_dotenv()

DB_PATH = os.getenv("SILVER_DUCKDB_PATH", "")
TMP_DIR = "/tmp/duckdb_tmp"  # make sure it exists

conn = duckdb.connect(DB_PATH)

conn.execute(f"SET temp_directory='{TMP_DIR}'")
conn.execute("SET memory_limit='1GB'")
conn.execute("SET threads=1")
conn.execute("SET preserve_insertion_order=false")

%sql conn --alias duck

Tip: You may define configurations in /home/azureuser/tramita/pyproject.toml or /home/azureuser/.jupysql/config.

Did not find user configurations in /home/azureuser/tramita/pyproject.toml.

## Funções, macros

In [2]:
import json
from pathlib import Path
import pyarrow as pa
import pyarrow.dataset as ds


def peek_json_payloads(
    root: str = "data/bronze/snapshots/bronze-2020-smoke",
    entity: str = "camara/proposicoes/details",
    *,
    n: int = 5,
    year: int | None = None,
) -> list[dict]:
    """
    Stream up to `n` JSON payloads from a hive-partitioned parquet dataset.
    Works across PyArrow versions (no scanner(limit=...) / to_table(limit=...)).
    """
    base = Path(root).as_posix().rstrip("/")
    path = f"{base}/{entity.strip('/')}"
    dataset = ds.dataset(path, format="parquet", partitioning="hive")

    names = set(dataset.schema.names)
    col = "payload_json" if "payload_json" in names else ("payload" if "payload" in names else None)
    if col is None:
        raise ValueError(f"No payload column found at {path}. Expected 'payload_json' or 'payload'.")

    flt = (ds.field("year") == year) if (year is not None and "year" in names) else None

    # Build a scanner without a limit and stream batches
    scanner = ds.Scanner.from_dataset(dataset, columns=[col], filter=flt)

    out: list[dict] = []
    for batch in scanner.to_batches():  # stream; no memory blow-up
        arr = batch.column(0)
        # slice if this batch would push us over n
        room = n - len(out)
        if room <= 0:
            break
        if len(arr) > room:
            arr = arr.slice(0, room)

        for v in arr.to_pylist():
            if v is None:
                continue
            if isinstance(v, (str, bytes, bytearray)):
                s = v.decode("utf-8") if isinstance(v, (bytes, bytearray)) else v
                try:
                    out.append(json.loads(s))
                except Exception:
                    out.append({"_raw": s})
            elif isinstance(v, dict):
                out.append(v)
            else:
                out.append({"_raw": v})

        if len(out) >= n:
            break

    # Pretty-print for quick inspection
    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out


In [3]:
from __future__ import annotations

import json
from pathlib import Path
import pyarrow.dataset as ds


def peek_rows(
    root: str = "data/bronze/snapshots/bronze-2020-smoke",
    entity: str = "camara/proposicoes/details",
    *,
    n: int = 5,
    year: int | None = None,
) -> list[dict]:
    """
    Stream up to `n` full rows (all columns) from a hive-partitioned parquet dataset.
    Converts Arrow rows to dicts for easy inspection.
    """
    base = Path(root).as_posix().rstrip("/")
    path = f"{base}/{entity.strip('/')}"
    dataset = ds.dataset(path, format="parquet", partitioning="hive")

    flt = (ds.field("year") == year) if (year is not None and "year" in dataset.schema.names) else None

    scanner = ds.Scanner.from_dataset(dataset, filter=flt)

    out: list[dict] = []
    for batch in scanner.to_batches():
        room = n - len(out)
        if room <= 0:
            break

        tbl = batch.to_pydict()
        # batch.to_pydict() gives {col: list}, need row-wise dicts
        num_rows = min(len(next(iter(tbl.values()))), room)
        for i in range(num_rows):
            row = {k: v[i] for k, v in tbl.items()}
            # Try to parse payload_json nicely if present
            if "payload_json" in row and isinstance(row["payload_json"], str):
                try:
                    row["payload_json"] = json.loads(row["payload_json"])
                except Exception:
                    pass
            out.append(row)

        if len(out) >= n:
            break

    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out


In [4]:
%%sql

CREATE OR REPLACE MACRO jget1(j, p) AS json_extract_string(j, p);
CREATE OR REPLACE MACRO jcoalesce(j, p1, p2, p3, p4, p5, p6) AS
  COALESCE(jget1(j, p1), jget1(j, p2), jget1(j, p3), jget1(j, p4), jget1(j, p5), jget1(j, p6));

Running query in 'duck'

Count


# Câmara dos Deputados

## Modelos dimensionais

### Blocos

In [5]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_blocos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/blocos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'blocos';

SELECT COUNT(*) AS n FROM bronze_camara_blocos;


Running query in 'duck'

n
5


In [6]:
%%sql
DROP TABLE IF EXISTS blocos_camara;
CREATE TABLE blocos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_blocos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_bloco,
    jget1(j, '$.dados.nome') AS nome,
    CAST(jget1(j, '$.dados.idLegislatura') AS BIGINT) AS id_legislatura,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_blocos;

Running query in 'duck'

Success


In [7]:
%%sql
SELECT * FROM blocos_camara;

Running query in 'duck'

id_bloco,nome,id_legislatura,uri,year_snapshot
584,Federação Brasil da Esperança - Fe Brasil,57,https://dadosabertos.camara.leg.br/api/v2/blocos/584,0
585,Federação PSDB CIDADANIA,57,https://dadosabertos.camara.leg.br/api/v2/blocos/585,0
586,Federação PSOL REDE,57,https://dadosabertos.camara.leg.br/api/v2/blocos/586,0
589,"PL, UNIÃO, PP, PSD, REPUBLICANOS, MDB, Federação PSDB CIDADANIA, PODE",57,https://dadosabertos.camara.leg.br/api/v2/blocos/589,0
590,"AVANTE, SOLIDARIEDADE, PRD",57,https://dadosabertos.camara.leg.br/api/v2/blocos/590,0


### Deputados

In [8]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/deputados/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados';

SELECT COUNT(*) AS n FROM bronze_camara_deputados;

Running query in 'duck'

n
3615


In [9]:
%%sql
SELECT * FROM bronze_camara_deputados LIMIT 10;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,deputados,62881,https://dadosabertos.camara.leg.br/api/v2/deputados/62881,"{""dados"":{""cpf"":""12133728368"",""dataFalecimento"":null,""dataNascimento"":""1958-09-01"",""escolaridade"":""Superior"",""id"":62881,""municipioNascimento"":""Fortaleza"",""nomeCivil"":""FRANCISCO DANILO BASTOS FORTE"",""redeSocial"":[""https://twitter.com/depdaniloforte"",""https://www.facebook.com/depdaniloforte"",""https://www.instagram.com/depdaniloforte""],""sexo"":""M"",""ufNascimento"":""CE"",""ultimoStatus"":{""condicaoEleitoral"":""Titular"",""data"":""2023-02-01"",""descricaoStatus"":null,""email"":null,""gabinete"":{""andar"":""2"",""email"":""dep.daniloforte@camara.leg.br"",""nome"":""219"",""predio"":""4"",""sala"":""219"",""telefone"":""3215-5219""},""id"":62881,""idLegislatura"":57,""nome"":""Danilo Forte"",""nomeEleitoral"":""Danilo Forte"",""siglaPartido"":""UNIÃO"",""siglaUf"":""CE"",""situacao"":""Exercício"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/62881"",""uriPartido"":null,""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/62881.jpg""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/62881"",""urlWebsite"":null}}",e5157e11c4ce6aee6ffad84ed23f8438760bfa2cd3445d768e942855e3d67764,0
camara,deputados,66385,https://dadosabertos.camara.leg.br/api/v2/deputados/66385,"{""dados"":{""cpf"":""77309766768"",""dataFalecimento"":null,""dataNascimento"":""1966-02-14"",""escolaridade"":""Superior"",""id"":66385,""municipioNascimento"":""Teresina"",""nomeCivil"":""JULIO FERRAZ ARCOVERDE"",""redeSocial"":[""https://twitter.com/julioarcoverde"",""https://www.facebook.com/depjulioarcoverde"",""https://www.instagram.com/julioarcoverde"",""https://youtube.com/julioarcoverde""],""sexo"":""M"",""ufNascimento"":""PI"",""ultimoStatus"":{""condicaoEleitoral"":""Titular"",""data"":""2023-02-01"",""descricaoStatus"":null,""email"":null,""gabinete"":{""andar"":""7"",""email"":""dep.julioarcoverde@camara.leg.br"",""nome"":""729"",""predio"":""4"",""sala"":""729"",""telefone"":""3215-5729""},""id"":66385,""idLegislatura"":57,""nome"":""Julio Arcoverde"",""nomeEleitoral"":""Julio Arcoverde"",""siglaPartido"":""PP"",""siglaUf"":""PI"",""situacao"":""Exercício"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/66385"",""uriPartido"":null,""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/66385.jpg""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/66385"",""urlWebsite"":null}}",22e9eaf23222140fa4a0eb9bccc115b0198dc34f2ea190276f43a2e89b20821e,0
camara,deputados,66828,https://dadosabertos.camara.leg.br/api/v2/deputados/66828,"{""dados"":{""cpf"":""28022995819"",""dataFalecimento"":null,""dataNascimento"":""1977-06-01"",""escolaridade"":""Superior"",""id"":66828,""municipioNascimento"":""Fernandópolis"",""nomeCivil"":""FAUSTO RUY PINATO"",""redeSocial"":[],""sexo"":""M"",""ufNascimento"":""SP"",""ultimoStatus"":{""condicaoEleitoral"":""Titular"",""data"":""2023-02-01"",""descricaoStatus"":null,""email"":null,""gabinete"":{""andar"":""5"",""email"":""dep.faustopinato@camara.leg.br"",""nome"":""562"",""predio"":""4"",""sala"":""562"",""telefone"":""3215-5562""},""id"":66828,""idLegislatura"":57,""nome"":""Fausto Pinato"",""nomeEleitoral"":""Fausto Pinato"",""siglaPartido"":""PP"",""siglaUf"":""SP"",""situacao"":""Exercício"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/66828"",""uriPartido"":null,""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/66828.jpg""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/66828"",""urlWebsite"":null}}",1554bfed21bd062d1b9df56b0d9791e2f64b4681d5a68861feb6a09375d24657,0
camara,deputados,69871,https://dadosabertos.camara.leg.br/api/v2/deputados/69871,"{""dados"":{""cpf"":""10626409500"",""dataFalecimento"":null,""dataNascimento"":""1957-07-09"",""escolaridade"":""Pós-Graduação"",""id"":69871,""municipioNascimento"":""Esplanada"",""nomeCivil""

In [10]:
%%sql
DROP TABLE IF EXISTS deputados_camara;
CREATE TABLE deputados_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_deputados
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_deputado,
    jget1(j, '$.dados.nomeCivil') AS nome_civil,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_deputados;

Running query in 'duck'

Success


### Eventos

In [11]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/eventos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos';

SELECT COUNT(*) AS n FROM bronze_camara_eventos;

Running query in 'duck'

n
1414


In [12]:

%%sql
DROP TABLE IF EXISTS eventos_camara;
CREATE TABLE eventos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_eventos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_evento,
    CAST(jget1(j, '$.dados.dataHoraInicio') AS DATETIME) AS data_hora_inicio,
    CAST(jget1(j, '$.dados.dataHoraFim') AS DATETIME) AS data_hora_fim,
    jget1(j, '$.dados.descricao') AS descricao,
    jget1(j, '$.dados.descricaoTipo') AS descricao_tipo,
    jget1(j, '$.dados.fases') AS fases,  -- é sempre null mas deixei por precaução
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_eventos;

Running query in 'duck'

Success


### Frentes

In [13]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_frentes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/frentes/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'frentes';

SELECT COUNT(*) AS n FROM bronze_camara_frentes;

Running query in 'duck'

n
1564


In [14]:
%%sql
DROP TABLE IF EXISTS frentes_camara;
CREATE TABLE frentes_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_frentes
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_frente,
    CAST(jget1(j, '$.dados.coordenador.id') AS BIGINT) AS id_deputado_coordenador,
    CAST(jget1(j, '$.dados.idLegislatura') AS BIGINT) AS id_legislatura,
    jget1(j, '$.dados.titulo') AS titulo,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_frentes;

Running query in 'duck'

Success


In [15]:
%%sql
SELECT * FROM frentes_camara LIMIT 5;

Running query in 'duck'

id_frente,id_deputado_coordenador,id_legislatura,titulo,uri,year_snapshot
346,160555,54,Frente Parlamentar pela criação do Ministério da Segurança Pública,https://dadosabertos.camara.leg.br/api/v2/frentes/346,0
347,74210,54,Frente Parlamentar Ambientalista,https://dadosabertos.camara.leg.br/api/v2/frentes/347,0
348,160533,54,Frente Parlamentar em Defesa dos Direitos da Criança e do Adolescente,https://dadosabertos.camara.leg.br/api/v2/frentes/348,0
350,141428,54,Frente Parlamentar Mista de Combate ao Crack,https://dadosabertos.camara.leg.br/api/v2/frentes/350,0
351,160646,54,Frente Parlamentar da Segurança Pública,https://dadosabertos.camara.leg.br/api/v2/frentes/351,0


### Legislaturas

In [16]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_legislaturas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/legislaturas/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'legislaturas';

SELECT COUNT(*) AS n FROM bronze_camara_legislaturas;

Running query in 'duck'

n
57


In [17]:

%%sql
DROP TABLE IF EXISTS legislaturas_camara;
CREATE TABLE legislaturas_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_legislaturas
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_legislatura,
    CAST(jget1(j, '$.dados.dataInicio') AS DATE) AS data_inicio,
    CAST(jget1(j, '$.dados.dataFim') AS DATE) AS data_fim,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_legislaturas;


Running query in 'duck'

Success


In [18]:
%%sql
SELECT * FROM legislaturas_camara;

Running query in 'duck'

id_legislatura,data_inicio,data_fim,uri,year_snapshot
1,1826-04-29,1830-04-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/1,0
2,1830-04-25,1834-04-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/2,0
3,1834-04-25,1838-04-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/3,0
4,1838-04-25,1842-04-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/4,0
5,1842-04-25,1844-05-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/5,0
6,1844-12-24,1848-04-24,https://dadosabertos.camara.leg.br/api/v2/legislaturas/6,0
7,1848-04-25,1849-02-19,https://dadosabertos.camara.leg.br/api/v2/legislaturas/7,0
8,1849-12-15,1853-04-14,https://dadosabertos.camara.leg.br/api/v2/legislaturas/8,0
9,1853-04-15,1857-04-14,https://dadosabertos.camara.leg.br/api/v2/legislaturas/9,0
10,1857-04-15,1861-04-14,https://dadosabertos.camara.leg.br/api/v2/legislaturas/10,0


### Órgaos

In [19]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/orgaos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_orgaos;


Running query in 'duck'

n
1822


In [20]:
%%sql
SELECT * FROM bronze_camara_orgaos LIMIT 10;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,orgaos,4,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,"{""dados"":{""apelido"":""Mesa Diretora"",""casa"":"""",""codTipoOrgao"":1,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":null,""dataInstalacao"":null,""id"":4,""nome"":""Mesa Diretora da Câmara dos Deputados"",""nomePublicacao"":""Mesa Diretora"",""nomeResumido"":null,""sala"":""Sala de Reuniões do Espaço Cultural"",""sigla"":""MESA"",""tipoOrgao"":""Comissão Diretora"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/4"",""urlWebsite"":null}}",681d0b88b7c289a596b29818f85525063e51492c3a3b63a5d521fefbf9d10476,0
camara,orgaos,111,https://dadosabertos.camara.leg.br/api/v2/orgaos/111,"{""dados"":{""apelido"":""CONGRESSO NACIONAL"",""casa"":"""",""codTipoOrgao"":81001,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":""2020-03-20T00:00"",""dataInstalacao"":""2020-03-20T00:00"",""id"":111,""nome"":""CONGRESSO NACIONAL"",""nomePublicacao"":""CN"",""nomeResumido"":""Congresso Nacional"",""sala"":""Plenário da Câmara dos Deputados"",""sigla"":""CN"",""tipoOrgao"":""Órgão do Congresso Nacional"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/111"",""urlWebsite"":null}}",2858236d7f6c5401b61e60c7d92c94166c6181b5d9341bd7248eaaf942657362,0
camara,orgaos,180,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,"{""dados"":{""apelido"":""Plenário"",""casa"":"""",""codTipoOrgao"":26,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":""2020-03-20T00:00"",""dataInstalacao"":""2020-03-20T00:00"",""id"":180,""nome"":""Plenário"",""nomePublicacao"":""Plenário"",""nomeResumido"":""Plenário"",""sala"":""Plenário da Câmara dos Deputados"",""sigla"":""PLEN"",""tipoOrgao"":""Plenário Virtual"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/180"",""urlWebsite"":null}}",a7fa8343decdf82e8b69812839bec2fcc1a3d9b4211e398c12802f49970759a2,0
camara,orgaos,186,https://dadosabertos.camara.leg.br/api/v2/orgaos/186,"{""dados"":{""apelido"":""COORDENAÇÃO DE COMISSÕES PERMANENTES"",""casa"":"""",""codTipoOrgao"":12000,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":null,""dataInstalacao"":null,""id"":186,""nome"":""COORDENAÇÃO DE COMISSÕES PERMANENTES"",""nomePublicacao"":""COORDENAÇÃO DE COMISSÕES PERMANENTES"",""nomeResumido"":null,""sala"":null,""sigla"":""CCP"",""tipoOrgao"":""Órgão da Câmara dos Deputados"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/186"",""urlWebsite"":null}}",0cf758d6db9b7954b6bc8fa87cc69d842e95313f7db06fcdee8f9bf412fe8162,0
camara,orgaos,209,https://dadosabertos.camara.leg.br/api/v2/orgaos/209,"{""dados"":{""apelido"":""PODER CONCLUSIVO NAS COMISSÕES"",""casa"":"""",""codTipoOrgao"":12000,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":null,""dataInstalacao"":null,""id"":209,""nome"":""PODER CONCLUSIVO NAS COMISSÕES"",""nomePublicacao"":""PODER CONCLUSIVO NAS COMISSÕES"",""nomeResumido"":null,""sala"":null,""sigla"":""PTCOM"",""tipoOrgao"":""Órgão da Câmara dos Deputados"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/209"",""urlWebsite"":null}}",d2473bfa9138c669645a3c842c26718ba79433d32ece0acb851bffa1ed0a911a,0
camara,orgaos,249,https://dadosabertos.camara.leg.br/api/v2/orgaos/249,"{""dados"":{""apelido"":""PRESIDÊNCIA DA CÂMARA DOS DEPUTADOS"",""casa"":"""",""codTipoOrgao"":12000,""dataFim"":null,""dataFimOriginal"":null,""dataInicio"":null,""dataInstalacao"":null,""id"":249,""nome"":""PRESIDÊNCIA DA CÂMARA DOS DEPUTADOS"",""nomePublicacao"":""PRESIDÊNCIA DA CÂMARA DOS DEPUTADOS"",""nomeResumido"":null,""sala"":null,""sigla"":""PC"",""tipoOrgao"":""Órgão da Câmara dos Deputados"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/249"",""urlWebsite"":null}}",80263bbf10c810084c1902eb71c656cd8513794383974e3990f7dbb64e4b9911,0
camara,orgaos,301,https://dadosabertos.camara.leg.br/api/v2/orgaos/301,"{""dados"":{""apelido"":""CONSELHO DE MINISTROS"",""casa"":"""",""codTipoOrgao"":12000,""dataFim"":null,""dataFimOrig

In [21]:

%%sql
DROP TABLE IF EXISTS orgaos_camara;
CREATE TABLE orgaos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_orgaos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_orgao,
    jget1(j, '$.dados.nome') AS nome,
    CAST(jget1(j, '$.dados.codTipoOrgao') AS BIGINT) AS cod_tipo_orgao,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_orgaos;


Running query in 'duck'

Success


In [22]:
%%sql
SELECT * FROM orgaos_camara LIMIT 10;

Running query in 'duck'

id_orgao,nome,cod_tipo_orgao,uri,year_snapshot
4,Mesa Diretora da Câmara dos Deputados,1,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,0
111,CONGRESSO NACIONAL,81001,https://dadosabertos.camara.leg.br/api/v2/orgaos/111,0
180,Plenário,26,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,0
186,COORDENAÇÃO DE COMISSÕES PERMANENTES,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/186,0
209,PODER CONCLUSIVO NAS COMISSÕES,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/209,0
249,PRESIDÊNCIA DA CÂMARA DOS DEPUTADOS,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/249,0
301,CONSELHO DE MINISTROS,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/301,0
302,PCM,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/302,0
325,MIC,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/325,0
340,Consultoria de Orçamento e Fiscalização Financeira da Câmara dos Deputados,12000,https://dadosabertos.camara.leg.br/api/v2/orgaos/340,0


### Partidos

In [23]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_partidos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/partidos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos';

SELECT COUNT(*) AS n FROM bronze_camara_partidos;

Running query in 'duck'

n
20


In [24]:
%%sql
SELECT * FROM bronze_camara_partidos LIMIT 5;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,partidos,36779,https://dadosabertos.camara.leg.br/api/v2/partidos/36779,"{""dados"":{""id"":36779,""nome"":""Partido Comunista do Brasil"",""numeroEleitoral"":null,""sigla"":""PCdoB"",""status"":{""data"":""2021-02-09T17:31"",""idLegislatura"":""57"",""lider"":{""idLegislatura"":57,""nome"":""Renildo Calheiros"",""siglaPartido"":""PCdoB"",""uf"":""PE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/73801"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/73801.jpg""},""situacao"":""Ativo"",""totalMembros"":""9"",""totalPosse"":""7"",""uriMembros"":""https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57&siglaPartido=PCdoB""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFacebook"":null,""urlLogo"":""https://www.camara.leg.br/internet/Deputado/img/partidos/PCdoB.gif"",""urlWebSite"":null}}",04756929e12e6adc200f5a32c401af0e9a10ca4ba355728999023e7fa1bc2572,0
camara,partidos,36786,https://dadosabertos.camara.leg.br/api/v2/partidos/36786,"{""dados"":{""id"":36786,""nome"":""Partido Democrático Trabalhista"",""numeroEleitoral"":null,""sigla"":""PDT"",""status"":{""data"":""2025-02-04T12:10"",""idLegislatura"":""57"",""lider"":{""idLegislatura"":57,""nome"":""Mário Heringer"",""siglaPartido"":""PDT"",""uf"":""MG"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/74158"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36786"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/74158.jpg""},""situacao"":""Ativo"",""totalMembros"":""16"",""totalPosse"":""16"",""uriMembros"":""https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57&siglaPartido=PDT""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36786"",""urlFacebook"":null,""urlLogo"":""https://www.camara.leg.br/internet/Deputado/img/partidos/PDT.gif"",""urlWebSite"":null}}",600d7224db7f8feca2f74e728e3a8d8f31e776ca7e0aacc2e10209e127bf7025,0
camara,partidos,36832,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,"{""dados"":{""id"":36832,""nome"":""Partido Socialista Brasileiro"",""numeroEleitoral"":null,""sigla"":""PSB"",""status"":{""data"":""2025-01-31T09:02"",""idLegislatura"":""57"",""lider"":{""idLegislatura"":57,""nome"":""Pedro Campos"",""siglaPartido"":""PSB"",""uf"":""PE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/220667"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36832"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/220667.jpg""},""situacao"":""Ativo"",""totalMembros"":""16"",""totalPosse"":""15"",""uriMembros"":""https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57&siglaPartido=PSB""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36832"",""urlFacebook"":null,""urlLogo"":""https://www.camara.leg.br/internet/Deputado/img/partidos/PSB.gif"",""urlWebSite"":null}}",5db173484bc7f2296e9bf3794097d673ebaae6b2d33e619b62142c8954b33ebd,0
camara,partidos,36834,https://dadosabertos.camara.leg.br/api/v2/partidos/36834,"{""dados"":{""id"":36834,""nome"":""Partido Social Democrático"",""numeroEleitoral"":null,""sigla"":""PSD"",""status"":{""data"":""2024-11-13T09:05"",""idLegislatura"":""57"",""lider"":{""idLegislatura"":57,""nome"":""Antonio Brito"",""siglaPartido"":""PSD"",""uf"":""BA"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/160553"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36834"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/160553.jpg""},""situacao"":""Ativo"",""totalMembros"":""45"",""totalPosse"":""42"",""uriMembros"":""https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57&siglaPartido=PSD""},""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36834"",""urlFacebook"":null,""urlLogo"":""https://www.camara.leg.br/in

In [25]:

%%sql
DROP TABLE IF EXISTS partidos_camara;
CREATE TABLE partidos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_partidos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_partido,
    jget1(j, '$.dados.nome') AS nome,
    jget1(j, '$.dados.sigla') AS sigla,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_partidos;


Running query in 'duck'

Success


In [26]:
%%sql
SELECT * from partidos_camara LIMIT 5;

Running query in 'duck'

id_partido,nome,sigla,uri,year_snapshot
36779,Partido Comunista do Brasil,PCdoB,https://dadosabertos.camara.leg.br/api/v2/partidos/36779,0
36786,Partido Democrático Trabalhista,PDT,https://dadosabertos.camara.leg.br/api/v2/partidos/36786,0
36832,Partido Socialista Brasileiro,PSB,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,0
36834,Partido Social Democrático,PSD,https://dadosabertos.camara.leg.br/api/v2/partidos/36834,0
36835,Partido da Social Democracia Brasileira,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,0


### Proposições

In [27]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_proposicoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/proposicoes/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'proposicoes';

SELECT COUNT(*) AS n FROM bronze_camara_proposicoes;


Running query in 'duck'

n
40435


In [28]:
%%sql
DROP TABLE IF EXISTS proposicoes_camara;
CREATE TABLE proposicoes_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_proposicoes
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_proposicao,
    jget1(j, '$.dados.siglaTipo') as sigla_tipo,
    CAST(jget1(j, '$.dados.numero') AS INTEGER) AS numero,
    CAST(jget1(j, '$.dados.ano') AS INTEGER) AS ano,
    jget1(j, '$.dados.ementa') AS ementa,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_proposicoes;

Running query in 'duck'

Success


In [29]:
%%sql

SELECT COUNT(*) AS n FROM proposicoes_camara;
SELECT * FROM proposicoes_camara ORDER BY ano DESC, numero DESC LIMIT 5;
SELECT COUNT(*) AS ids, COUNT(DISTINCT id_proposicao) AS distinct_ids FROM proposicoes_camara;


Running query in 'duck'

ids,distinct_ids
40435,40435


### Temas

In [30]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_temas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/temas/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'temas';

SELECT COUNT(*) AS n FROM bronze_camara_temas;

Running query in 'duck'

n
40435


In [31]:
%%sql
SELECT * FROM bronze_camara_temas LIMIT 5;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,temas,15009,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15009/temas,"{""dados"":[{""codTema"":42,""relevancia"":0,""tema"":""Direito Civil e Processual Civil""},{""codTema"":43,""relevancia"":0,""tema"":""Direito Penal e Processual Penal""}]}",932de0161d42aa46b09c358116b7120be5a6aff80493d0225b5f8de5729b6e8d,2020
camara,temas,15532,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15532/temas,"{""dados"":[{""codTema"":40,""relevancia"":0,""tema"":""Economia""},{""codTema"":61,""relevancia"":0,""tema"":""Viação, Transporte e Mobilidade""}]}",fb310f4af301f7e3d8ab7eae2f10ca310b0d232a4156cd7b74db1bea12685828,2020
camara,temas,15749,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15749/temas,"{""dados"":[{""codTema"":42,""relevancia"":0,""tema"":""Direito Civil e Processual Civil""}]}",a04fb1504594e1b2e9c45e3a5bc6a2fbd30419adf1b272740e0f762a1151c8b2,2020
camara,temas,15781,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15781/temas,"{""dados"":[{""codTema"":42,""relevancia"":0,""tema"":""Direito Civil e Processual Civil""}]}",a04fb1504594e1b2e9c45e3a5bc6a2fbd30419adf1b272740e0f762a1151c8b2,2020
camara,temas,15990,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15990/temas,"{""dados"":[{""codTema"":62,""relevancia"":0,""tema"":""Ciência, Tecnologia e Inovação""},{""codTema"":54,""relevancia"":0,""tema"":""Energia, Recursos Hídricos e Minerais""}]}",0fe4b6544b008625110c85a8e30d1be38d5746cc79484b305b8f330a0d71936f,2020


In [32]:
%%sql
DROP TABLE IF EXISTS temas_camara;
CREATE TABLE temas_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_temas
),
exploded as (
    SELECT
        CAST(jget1(elem.value, '$.codTema') AS BIGINT) AS id_tema,
        jget1(elem.value, '$.tema') AS descricao,
    FROM base
    CROSS JOIN json_each(json_extract(j, '$.dados')) AS elem
)
SELECT
    id_tema,
    max(descricao) as descricao,
FROM exploded
WHERE id_tema IS NOT NULL
GROUP BY id_tema
ORDER BY id_tema;
DROP VIEW IF EXISTS bronze_camara_temas;

Running query in 'duck'

Success


In [33]:
%%sql
SELECT * FROM temas_camara LIMIT 5;

Running query in 'duck'

id_tema,descricao
34,Administração Pública
35,"Arte, Cultura e Religião"
37,Comunicações
39,Esporte e Lazer
40,Economia


## Modelos factuais

### Autores

In [34]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_autores AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/autores/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'autores';

SELECT COUNT(*) AS n FROM bronze_camara_autores;

Running query in 'duck'

n
40435


In [35]:
%%sql
DROP TABLE IF EXISTS autores_camara;
CREATE TABLE autores_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j,
    year,
    id AS id_proposicao,
    FROM bronze_camara_autores
),
exploded as (
    SELECT
        CAST(jget1(elem.value, '$.codTipo') AS BIGINT) AS cod_tipo,
        jget1(elem.value, '$.uri') AS uri,
        CAST(jget1(elem.value, '$.ordemAssinatura') AS INTEGER) AS ordem_assinatura,
        CAST(jget1(elem.value, '$.proponente') AS BOOLEAN) AS proponente,
        CAST(b.id_proposicao AS BIGINT) as id_proposicao,
        b.year
    FROM base b
    CROSS JOIN json_each(b.j, '$.dados') AS elem
)
SELECT
    cod_tipo, uri, ordem_assinatura, proponente, id_proposicao, year
FROM exploded;
DROP VIEW IF EXISTS bronze_camara_autores;

Running query in 'duck'

Success


In [36]:
%%sql
SELECT * FROM autores_camara LIMIT 10;

Running query in 'duck'

cod_tipo,uri,ordem_assinatura,proponente,id_proposicao,year
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/74316,1,True,15749,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/74295,1,True,15781,2020
20,https://dadosabertos.camara.leg.br/api/v2/orgaos/262,1,True,15990,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/73442,1,True,16078,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/73899,1,True,16579,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/74779,1,True,16792,2020
40000,https://dadosabertos.camara.leg.br/api/v2/orgaos/78,1,True,16969,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/73896,1,True,17495,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/74419,1,True,17496,2020
10000,https://dadosabertos.camara.leg.br/api/v2/deputados/74665,1,True,17593,2020


### Orientações

In [37]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_orientacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/orientacoes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'orientacoes';

SELECT COUNT(*) AS n FROM bronze_camara_orientacoes;

Running query in 'duck'

n
9099


In [38]:
%%sql
SELECT * FROM bronze_camara_orientacoes LIMIT 5;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,orientacoes,1006391-32,https://dadosabertos.camara.leg.br/api/v2/votacoes/1006391-32/orientacoes,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,orientacoes,103453-26,https://dadosabertos.camara.leg.br/api/v2/votacoes/103453-26/orientacoes,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,orientacoes,1046197-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/1046197-43/orientacoes,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,orientacoes,1048959-19,https://dadosabertos.camara.leg.br/api/v2/votacoes/1048959-19/orientacoes,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,orientacoes,1048959-27,https://dadosabertos.camara.leg.br/api/v2/votacoes/1048959-27/orientacoes,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020


In [39]:
%%sql
DROP TABLE IF EXISTS orientacoes_camara;

CREATE TABLE orientacoes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_votacao
  FROM bronze_camara_orientacoes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.codPartidoBloco') AS BIGINT) AS cod_partido_bloco,
    json_extract_string(e.value, '$.codTipoLideranca') AS cod_tipo_lideranca,
    json_extract_string(e.value, '$.orientacaoVoto') AS orientacao_voto,
    json_extract_string(e.value, '$.siglaPartidoBloco') AS sigla_partido_bloco,
    json_extract_string(e.value, '$.uriPartidoBloco') AS uri_partido_bloco,
    b.id_votacao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_votacao,
    sigla_partido_bloco,
    orientacao_voto,
    cod_partido_bloco,
    cod_tipo_lideranca,
    uri_partido_bloco,
    year_snapshot,
FROM exploded;
DROP VIEW IF EXISTS bronze_camara_orientacoes;


Running query in 'duck'

Success


In [40]:
%%sql
SELECT * FROM orientacoes_camara LIMIT 5;

Running query in 'duck'

id_votacao,sigla_partido_bloco,orientacao_voto,cod_partido_bloco,cod_tipo_lideranca,uri_partido_bloco,year_snapshot
105464-262,NOVO,Não,37901,P,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,2020
105464-319,PT,Sim,36844,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,2020
105464-326,NOVO,Não,37901,P,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,2020
1198512-250,PTB,Sim,36845,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36845,2020
1198512-254,DEM,Não,36769,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,2020


### Proposições relacionadas

In [41]:
import pyarrow.parquet as pq
import json
import pandas as pd
import tempfile
from glob import glob

paths = glob("data/bronze/snapshots/bronze-2020-smoke/camara/relacionadas/year=*/part-*.parquet")

with tempfile.NamedTemporaryFile() as tf:

    table = pq.read_table(paths)
    df = table.to_pandas()

    df.head()
    exploded_rows = []

    for index, row in df.iterrows():
        payload = json.loads(row['payload_json'])
        for item in payload['dados']:
            exploded_rows.append({
                'id_proposicao_origem': row['id'],
                'id_proposicao_destino': item['id'],
                'year_snapshot': row['year'],
            })
    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name)

    conn.execute(f"""
        DROP TABLE IF EXISTS relacionadas_camara;
        CREATE TABLE relacionadas_camara AS
            SELECT
                id_proposicao_origem::BIGINT AS id_proposicao_origem,
                id_proposicao_destino::BIGINT AS id_proposicao_destino,
                year_snapshot::INTEGER AS year_snapshot
            FROM read_csv_auto('{tf.name}', HEADER=TRUE);
    """)

In [42]:
%%sql
SELECT COUNT(*) FROM relacionadas_camara;

Running query in 'duck'

count_star()
75961


In [43]:
%%sql
SELECT * FROM relacionadas_camara LIMIT 5;

Running query in 'duck'

id_proposicao_origem,id_proposicao_destino,year_snapshot
15009,59509,2020
15009,146124,2020
15009,59511,2020
15009,83544,2020
15009,96371,2020


### Tramitações

In [44]:
import pyarrow.parquet as pq
import json
import pandas as pd
import tempfile
from glob import glob

with tempfile.NamedTemporaryFile() as tf:

    # load parquet
    paths = glob("data/bronze/snapshots/bronze-2020-smoke/camara/tramitacoes/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    exploded_rows = []

    for index, row in df.iterrows():
        payload = json.loads(row["payload_json"])
        for item in payload.get("dados", []):
            exploded_rows.append({
                "id_proposicao": row["id"],
                "ambito": item.get("ambito"),
                "apreciacao": item.get("apreciacao"),
                "cod_situacao": item.get("codSituacao"),
                "cod_tipo_tramitacao": item.get("codTipoTramitacao"),
                "data_hora": item.get("dataHora"),
                "descricao_situacao": item.get("descricaoSituacao"),
                "descricao_tramitacao": item.get("descricaoTramitacao"),
                "despacho": item.get("despacho"),
                "regime": item.get("regime"),
                "sequencia": item.get("sequencia"),
                "sigla_orgao": item.get("siglaOrgao"),
                "uri_orgao": item.get("uriOrgao"),
                "uri_ultimo_relator": item.get("uriUltimoRelator"),
                "year_snapshot": row["year"],
            })

    exploded_df = pd.DataFrame(exploded_rows)

    # save to temp CSV
    exploded_df.to_csv(tf.name, index=False)

    # create SQL table
    conn.execute(f"""
        DROP TABLE IF EXISTS tramitacoes_camara;
        CREATE TABLE tramitacoes_camara AS
            SELECT
                id_proposicao::BIGINT AS id_proposicao,
                ambito::TEXT AS ambito,
                apreciacao::TEXT AS apreciacao,
                cod_situacao::TEXT AS cod_situacao,
                cod_tipo_tramitacao::TEXT AS cod_tipo_tramitacao,
                CAST(data_hora AS TIMESTAMP) AS data_hora,
                descricao_situacao::TEXT AS descricao_situacao,
                descricao_tramitacao::TEXT AS descricao_tramitacao,
                despacho::TEXT AS despacho,
                regime::TEXT AS regime,
                sequencia::INTEGER AS sequencia,
                sigla_orgao::TEXT AS sigla_orgao,
                uri_orgao::TEXT AS uri_orgao,
                uri_ultimo_relator::TEXT AS uri_ultimo_relator,
                year_snapshot::INTEGER AS year_snapshot
            FROM read_csv_auto('{tf.name}', HEADER=TRUE);
    """)


In [45]:
%%sql

SELECT * FROM tramitacoes_camara LIMIT 10;

Running query in 'duck'

id_proposicao,ambito,apreciacao,cod_situacao,cod_tipo_tramitacao,data_hora,descricao_situacao,descricao_tramitacao,despacho,regime,sequencia,sigla_orgao,uri_orgao,uri_ultimo_relator,year_snapshot
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,100,1995-02-22 00:00:00,None,Apresentação de Proposição,APRESENTAÇÃO DO PROJETO PELO DEP JOSE COIMBRA.,"Prioridade (Art. 151, II, RICD)",1,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,105,1995-03-14 00:00:00,None,Leitura e publicação,LEITURA E PUBLICAÇÃO DA MATERIA. DCN1 24 03 95 PAG 4218 COL 01.,"Prioridade (Art. 151, II, RICD)",3,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,923.0,999,1995-03-14 00:00:00,Arquivada,Despacho* (inativa),DESPACHO INICIAL A CCJR.,"Prioridade (Art. 151, II, RICD)",2,MESA,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,None,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,180,1995-03-15 00:00:00,None,Encaminhamento,ENCAMINHADO A COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃO.,"Prioridade (Art. 151, II, RICD)",4,CCP,https://dadosabertos.camara.leg.br/api/v2/orgaos/186,None,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,320,1995-03-16 00:00:00,None,Designação de Relator(a),RELATOR DEP REGIS DE OLIVEIRA. DCN1 17 03 95 PAG 3668 COL 01.,"Prioridade (Art. 151, II, RICD)",5,CCJC,https://dadosabertos.camara.leg.br/api/v2/orgaos/2003,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,322,1995-03-29 10:03:00,None,Parecer do(a) Relator(a),"Parecer do Relator, Dep. Regis de Olieira, pela constitucionalidade, juridicidade e técnica legislativa, e, no mérito, pela aprovação deste, com emendas.","Prioridade (Art. 151, II, RICD)",6,CCJC,https://dadosabertos.camara.leg.br/api/v2/orgaos/2003,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,336,1995-05-10 00:00:00,None,Aprovação do Parecer,"APROVAÇÃO UNANIME DO PARECER DO RELATOR, DEP REGIS DEOLIVEIRA, PELA CONSTITUCIONALIDADE, JURIDICIDADE ETECNICA LEGISLATIVA E, NO MERITO, PELA APROVAÇÃO, COMEMENDAS.DCN1 27 06 95 PAG 14156 COL 02.","Prioridade (Art. 151, II, RICD)",7,CCJC,https://dadosabertos.camara.leg.br/api/v2/orgaos/2003,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,330,1995-06-02 00:00:00,None,Leitura e publicação do Parecer,LEITURA E PUBLICAÇÃO DO PARECER DA CCJR. PRONTO PARA A ORDEM DO DIA. PL. 70-A/95.,"Prioridade (Art. 151, II, RICD)",8,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,923.0,5,1995-06-29 00:00:00,Arquivada,Não Informado,"DEFERIDO REQUERIMENTO, DO DEP OSMANIO PEREIRA,SOLICITANDO AUDIENCIA DA CSSF PARA ESTE PROJETO.","Prioridade (Art. 151, II, RICD)",9,MESA,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020
15009,Regimental,Proposição Sujeita à Apreciação do Plenário,None,221,1995-06-29 00:00:00,None,Adiamento de Discussão,DISCUSSÃO EM TURNO UNICO. APROVAÇÃO DO REQUERIMENTO DO DEP SALVADOR ZIMBALDI (PSDB) SOLICITANDO O ADIAMENTO DA DISCUSSÃO POR 10 SESSÕES. DCN1 30 06 95 PAG 14667 COL 01.,"Prioridade (Art. 151, II, RICD)",10,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/deputados/73556,2020


### Votações

In [46]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_votacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/votacoes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'votacoes';

SELECT COUNT(*) AS n FROM bronze_camara_votacoes;

Running query in 'duck'

n
40435


In [47]:
%%sql
SELECT * FROM bronze_camara_votacoes LIMIT 10;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,votacoes,15009,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15009/votacoes,"{""dados"":[{""aprovacao"":1,""data"":""1996-04-10"",""dataHoraRegistro"":null,""descricao"":""APROVAÇÃO UNANIME DO PARECER FAVORAVEL DO RELATOR,\r\nDEP AGNELO QUEIROZ.\r\nDCDS 15 06 96 PAG 0418 COL 01."",""id"":""15009-13"",""proposicaoObjeto"":null,""siglaOrgao"":""CSAUDE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15009-13"",""uriEvento"":null,""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2014"",""uriProposicaoObjeto"":null},{""aprovacao"":1,""data"":""1995-05-10"",""dataHoraRegistro"":null,""descricao"":""APROVAÇÃO UNANIME DO PARECER DO RELATOR, DEP REGIS DE\r\nOLIVEIRA, PELA CONSTITUCIONALIDADE, JURIDICIDADE E\r\nTECNICA LEGISLATIVA E, NO MERITO, PELA APROVAÇÃO, COM\r\nEMENDAS.\r\nDCN1 27 06 95 PAG 14156 COL 02."",""id"":""15009-6"",""proposicaoObjeto"":null,""siglaOrgao"":""CCJC"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15009-6"",""uriEvento"":null,""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2003"",""uriProposicaoObjeto"":null}]}",670b605734371636c04ccea479375265b1ce7bdba13820386094b47ae81f71d0,2020
camara,votacoes,15532,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15532/votacoes,"{""dados"":[{""aprovacao"":1,""data"":""2002-04-10"",""dataHoraRegistro"":null,""descricao"":""Aprovado o Parecer contra os votos dos Deputados Saraiva Felipe e Arnaldo Faria de Sá"",""id"":""15532-63"",""proposicaoObjeto"":null,""siglaOrgao"":""CSAUDE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15532-63"",""uriEvento"":""https://dadosabertos.camara.leg.br/api/v2/eventos/2122"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2014"",""uriProposicaoObjeto"":null},{""aprovacao"":1,""data"":""2002-11-13"",""dataHoraRegistro"":null,""descricao"":""Aprovado por Unanimidade o Parecer"",""id"":""15532-75"",""proposicaoObjeto"":null,""siglaOrgao"":""CFT"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15532-75"",""uriEvento"":""https://dadosabertos.camara.leg.br/api/v2/eventos/2985"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2010"",""uriProposicaoObjeto"":null},{""aprovacao"":1,""data"":""2002-12-10"",""dataHoraRegistro"":null,""descricao"":""Aprovado por Unanimidade o Parecer"",""id"":""15532-88"",""proposicaoObjeto"":null,""siglaOrgao"":""CCJC"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15532-88"",""uriEvento"":""https://dadosabertos.camara.leg.br/api/v2/eventos/3169"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2003"",""uriProposicaoObjeto"":null}]}",9675f5f16a19d8f704f8e6d7d8335d3de03689c8f846652656f1c24214808462,2020
camara,votacoes,15749,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15749/votacoes,"{""dados"":[{""aprovacao"":1,""data"":""2002-03-13"",""dataHoraRegistro"":null,""descricao"":""Aprovado por Unanimidade o Parecer"",""id"":""15749-31"",""proposicaoObjeto"":null,""siglaOrgao"":""CDE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15749-31"",""uriEvento"":""https://dadosabertos.camara.leg.br/api/v2/eventos/1900"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2008"",""uriProposicaoObjeto"":null},{""aprovacao"":1,""data"":""2011-08-17"",""dataHoraRegistro"":""2011-08-17T11:09:17"",""descricao"":""Aprovado o Parecer, contra o voto do Deputado Eli Correa Filho."",""id"":""15749-97"",""proposicaoObjeto"":null,""siglaOrgao"":""CDC"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/votacoes/15749-97"",""uriEvento"":""https://dadosabertos.camara.leg.br/api/v2/eventos/26552"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/2004"",""uriProposicaoObjeto"":null}]}",60ac470156829a9d83c5716b336a4729f65c086d44a566b48090f170e4daa099,2020
camara,votacoes,15781,https://dadosabertos.camara.leg.br/api/v2/proposicoes/15781/votacoes,"{""dados"":[]}",e3f60f959291f4

In [48]:
%%sql
DROP TABLE IF EXISTS votacoes_camara;

CREATE TABLE votacoes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year as year_snapshot,
    id AS id_proposicao
  FROM bronze_camara_votacoes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.aprovacao') AS BOOLEAN) AS aprovacao,
    CAST(json_extract_string(e.value, '$.data') AS DATE) AS data,
    json_extract_string(e.value, '$.descricao') AS descricao,
    json_extract_string(e.value, '$.id') AS id_votacao, -- como text mesmo, por conta do hífen
    json_extract_string(e.value, '$.uri') AS uri,
    json_extract_string(e.value, '$.uriEvento') AS uri_evento,
    json_extract_string(e.value, '$.uriOrgao') AS uri_orgao,
    CAST(b.id_proposicao AS BIGINT) AS id_proposicao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_votacao,
    id_proposicao,
    data,
    descricao,
    aprovacao,
    uri_evento,
    uri_orgao,
    uri,
    year_snapshot,
FROM exploded;


Running query in 'duck'

Count
9272


In [49]:
%%sql
SELECT * FROM votacoes_camara LIMIT 10;

Running query in 'duck'

id_votacao,id_proposicao,data,descricao,aprovacao,uri_evento,uri_orgao,uri,year_snapshot
15009-6,15009,1995-05-10,"APROVAÇÃO UNANIME DO PARECER DO RELATOR, DEP REGIS DE OLIVEIRA, PELA CONSTITUCIONALIDADE, JURIDICIDADE E TECNICA LEGISLATIVA E, NO MERITO, PELA APROVAÇÃO, COM EMENDAS. DCN1 27 06 95 PAG 14156 COL 02.",True,None,https://dadosabertos.camara.leg.br/api/v2/orgaos/2003,https://dadosabertos.camara.leg.br/api/v2/votacoes/15009-6,2020
15532-88,15532,2002-12-10,Aprovado por Unanimidade o Parecer,True,https://dadosabertos.camara.leg.br/api/v2/eventos/3169,https://dadosabertos.camara.leg.br/api/v2/orgaos/2003,https://dadosabertos.camara.leg.br/api/v2/votacoes/15532-88,2020
15749-97,15749,2011-08-17,"Aprovado o Parecer, contra o voto do Deputado Eli Correa Filho.",True,https://dadosabertos.camara.leg.br/api/v2/eventos/26552,https://dadosabertos.camara.leg.br/api/v2/orgaos/2004,https://dadosabertos.camara.leg.br/api/v2/votacoes/15749-97,2020
16078-18,16078,1999-11-17,"APROVAÇÃO UNANIME DO PARECER FAVORAVEL DO RELATOR, DEP PAULO ROCHA, A ESTE E AOS PL. 3718/97 E PL. 4485/98, APENSADOS, COM SUBSTITUTIVO.",True,None,https://dadosabertos.camara.leg.br/api/v2/orgaos/2015,https://dadosabertos.camara.leg.br/api/v2/votacoes/16078-18,2020
16481-68,16481,2007-12-05,"Aprovado o Parecer com a abstenção dos Deputados Carlos Sampaio, Rita Camata, Marcelo Almeida e Marcelo Itagiba",True,https://dadosabertos.camara.leg.br/api/v2/eventos/16919,https://dadosabertos.camara.leg.br/api/v2/orgaos/5503,https://dadosabertos.camara.leg.br/api/v2/votacoes/16481-68,2020
597632-2,16969,2013-11-05,Aprovado com alteração. O Deputado Moreira Mendes sugeriu que os registro do Seminário e das oficinas citados no requerimento sejam para subsidiar a elaboração do Parecer. O autor acatou a sugestão e teve o apoio dos membros da Comissão.,True,https://dadosabertos.camara.leg.br/api/v2/eventos/34333,https://dadosabertos.camara.leg.br/api/v2/orgaos/8190,https://dadosabertos.camara.leg.br/api/v2/votacoes/597632-2,2020
17563-91,17563,2009-10-28,"Aprovado por Unanimidade o Parecer com Complementação de Voto, apresentou voto em separado o Deputado Dr. Rosinha.",True,https://dadosabertos.camara.leg.br/api/v2/eventos/22354,https://dadosabertos.camara.leg.br/api/v2/orgaos/2018,https://dadosabertos.camara.leg.br/api/v2/votacoes/17563-91,2020
2087878-2,17823,2016-06-15,Aprovado. Subscrito pelo deputado Silas Freire.,True,https://dadosabertos.camara.leg.br/api/v2/eventos/44095,https://dadosabertos.camara.leg.br/api/v2/orgaos/2014,https://dadosabertos.camara.leg.br/api/v2/votacoes/2087878-2,2020
469492-5,17915,2012-06-27,"Aprovado, por unanimidade, o Requerimento de Urgência (Art. 155 do RICD). DCD do dia 28/06/2012 PÁG 23339 COL 01.",True,https://dadosabertos.camara.leg.br/api/v2/eventos/29629,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/votacoes/469492-5,2020
373892-2,18420,2007-10-31,Aprovado.,True,https://dadosabertos.camara.leg.br/api/v2/eventos/16568,https://dadosabertos.camara.leg.br/api/v2/orgaos/2015,https://dadosabertos.camara.leg.br/api/v2/votacoes/373892-2,2020


### Votos

In [50]:
import pyarrow.parquet as pq
import json
import pandas as pd
import tempfile
from glob import glob

with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-smoke/camara/votos/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    df.head()
    exploded_rows = []

    for index, row in df.iterrows():
        payload = json.loads(row['payload_json'])
        for item in payload['dados']:
            exploded_rows.append({
                'id_votacao': row['id'],
                'id_deputado': item['deputado_']['id'],
                'tipo_voto': item['tipoVoto'],
                'data_hora': item['dataRegistroVoto'],
                'year_snapshot': row['year'],
            })
    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name)

    conn.execute(f"""
        DROP TABLE IF EXISTS votos_camara;
        CREATE TABLE votos_camara AS
            SELECT
                id_votacao::TEXT AS id_votacao,
                id_deputado::BIGINT AS id_deputado,
                tipo_voto::TEXT AS tipo_voto,
                CAST(data_hora AS TIMESTAMP) AS data_hora,
                year_snapshot::INTEGER AS year_snapshot
            FROM read_csv_auto('{tf.name}', HEADER=TRUE);
    """)

In [51]:
%%sql
SELECT * FROM votos_camara LIMIT 10;

Running query in 'duck'

id_votacao,id_deputado,tipo_voto,data_hora,year_snapshot
105464-262,62881,Não,2021-12-15 11:24:29,2020
105464-262,73466,Não,2021-12-15 11:24:29,2020
105464-262,73604,Sim,2021-12-15 11:24:29,2020
105464-262,74160,Sim,2021-12-15 11:24:29,2020
105464-262,116379,Não,2021-12-15 11:24:29,2020
105464-262,141411,Sim,2021-12-15 11:24:29,2020
105464-262,141488,Sim,2021-12-15 11:24:29,2020
105464-262,160575,Sim,2021-12-15 11:24:29,2020
105464-262,160673,Não,2021-12-15 11:24:29,2020
105464-262,178951,Sim,2021-12-15 11:24:29,2020


## Outras relações entre tabelas

### Blocos e partidos

In [52]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_blocos_partidos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/blocos/partidos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'blocos/partidos';

SELECT COUNT(*) AS n FROM bronze_camara_blocos_partidos;

Running query in 'duck'

n
5


In [53]:
%%sql
SELECT * FROM bronze_camara_blocos_partidos LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,blocos/partidos,584,https://dadosabertos.camara.leg.br/api/v2/blocos/584/partidos,"{""dados"":[{""id"":36779,""nome"":""Partido Comunista do Brasil"",""sigla"":""PCdoB"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779""},{""id"":36844,""nome"":""Partido dos Trabalhadores"",""sigla"":""PT"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36844""},{""id"":36851,""nome"":""Partido Verde"",""sigla"":""PV"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36851""}]}",c1402ccc6ef392ea8f2d6860b18bedd4e42344fd2f2d32ef33b05d9652db3320,2020


In [54]:
%%sql
DROP TABLE IF EXISTS blocos_partidos_camara;

CREATE TABLE blocos_partidos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_bloco
  FROM bronze_camara_blocos_partidos
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_partido,
    CAST(b.id_bloco AS BIGINT) AS id_bloco,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT id_bloco, id_partido, year_snapshot
FROM exploded;



Running query in 'duck'

Count
22


In [55]:
%%sql
SELECT * FROM blocos_partidos_camara;

Running query in 'duck'

id_bloco,id_partido,year_snapshot
584,36851,2020
585,36835,2020
586,36886,2020
589,38009,2020
590,37904,2020
584,36844,2020
585,37905,2020
586,36839,2020
589,37904,2020
590,38010,2020


### Deputados e frentes

In [56]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_frentes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/deputados/frentes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/frentes';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_frentes;

Running query in 'duck'

n
183


In [57]:
%%sql
SELECT * FROM bronze_camara_deputados_frentes LIMIT 1;

Running query in 'duck'

+--------+-------------------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
%%sql
DROP TABLE IF EXISTS deputados_frentes_camara;

CREATE TABLE deputados_frentes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_frentes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_frente,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT id_deputado, id_frente, year_snapshot
FROM exploded;


Running query in 'duck'

Count
21410


In [59]:
%%sql
SELECT * FROM deputados_frentes_camara LIMIT 10;

Running query in 'duck'

id_deputado,id_frente,year_snapshot
66385,54304,2020
122974,53890,2020
161440,54293,2020
178860,53457,2020
180214,54315,2020
204395,53890,2020
204406,53890,2020
204492,53892,2020
204524,53890,2020
220527,54368,2020


### Deputados e histórico

In [60]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_historico AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/deputados/historico/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/historico';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_historico;

Running query in 'duck'

n
183


In [61]:
%%sql
SELECT * FROM bronze_camara_deputados_historico LIMIT 1;

Running query in 'duck'

+--------+---------------------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [62]:
%%sql
DROP TABLE IF EXISTS deputados_historico_camara;

CREATE TABLE deputados_historico_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_historico
),
exploded AS (
  SELECT
    json_extract_string(e.value, '$.condicaoEleitoral') AS condicao_eleitoral,
    CAST(json_extract_string(e.value, '$.dataHora') AS DATETIME) AS data_hora,
    json_extract_string(e.value, '$.descricaoStatus') AS descricao_status,
    CAST(json_extract_string(e.value, '$.idLegislatura') AS BIGINT) AS id_legislatura,
    json_extract_string(e.value, '$.situacao') AS situacao,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_deputado,
    id_legislatura,
    data_hora,
    condicao_eleitoral,
    descricao_status,
    year_snapshot,
FROM exploded;


Running query in 'duck'

Count
500


In [63]:
%%sql
SELECT * FROM deputados_historico_camara LIMIT 5;

Running query in 'duck'

id_deputado,id_legislatura,data_hora,condicao_eleitoral,descricao_status,year_snapshot
178860,57,2023-02-01 12:05:00,Titular,Entrada - Posse de Eleito Titular - Posse na Sessão Preparatória,2015
122974,57,2023-02-01 12:05:00,Titular,Entrada - Posse de Eleito Titular - Posse na Sessão Preparatória,2019
204395,57,2023-02-01 12:05:00,Titular,Entrada - Posse de Eleito Titular - Posse na Sessão Preparatória,2019
204406,57,2023-02-01 12:05:00,Titular,Entrada - Posse de Eleito Titular - Posse na Sessão Preparatória,2019
204492,57,2023-02-01 12:05:00,Titular,Entrada - Posse de Eleito Titular - Posse na Sessão Preparatória,2019


### Deputados e órgãos

In [64]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/deputados/orgaos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_orgaos;

Running query in 'duck'

n
183


In [65]:
%%sql

SELECT * FROM bronze_camara_deputados_orgaos LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,deputados/orgaos,66385,https://dadosabertos.camara.leg.br/api/v2/deputados/66385/orgaos,"{""dados"":[{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2023-11-02T00:00"",""idOrgao"":539644,""nomeOrgao"":""Bancada Negra da Câmara dos Deputados, criada pela Resolução n. 6, de 2023, que acresceu ao Regimento Interno da Câmara dos Deputados o Capítulo VI, do Título I, com os artigos 13-A e 13-B"",""nomePublicacao"":null,""siglaOrgao"":""BANEGRA"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/539644""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2024-02-07T00:00"",""idOrgao"":539635,""nomeOrgao"":""Comissão Especial destinada a estudar e analisar formas de prevenção e auxílio a desastres e calamidades naturais que vêm assolando o Território Nacional"",""nomePublicacao"":""Comissão Especial sobre Prevenção e Auxílio a Desastres e Calamidades Naturais"",""siglaOrgao"":""CEDESNAT"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/539635""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2025-03-18T00:00"",""idOrgao"":6066,""nomeOrgao"":""Comissão de Turismo"",""nomePublicacao"":""Comissão de Turismo"",""siglaOrgao"":""CTUR"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/6066""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2025-03-18T00:00"",""idOrgao"":537236,""nomeOrgao"":""Comissão do Esporte"",""nomePublicacao"":""Comissão do Esporte"",""siglaOrgao"":""CESPO"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/537236""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2025-04-29T00:00"",""idOrgao"":539687,""nomeOrgao"":""Subcomissão Permanente de Regulação de Apostas Esportivas"",""nomePublicacao"":""Subcomissão Permanente de Regulação de Apostas Esportivas (CESPO/SUBAPOST)"",""siglaOrgao"":""SUBAPOST"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/539687""},{""codTitulo"":""102"",""dataFim"":null,""dataInicio"":""2025-04-29T00:00"",""idOrgao"":539771,""nomeOrgao"":""Comissão Especial destinada a proferir parecer ao Projeto de Lei nº 1087, de 2025, do Poder Executivo, que \""altera a legislação do imposto sobre a renda para instituir a redução do imposto devido nas bases de cálculo mensal e anual e a tributação mínima para as pessoas físicas que auferem altas rendas, e dá outras providências\"""",""nomePublicacao"":""Comissão Especial sobre Alteração da Legislação do Imposto de Renda (PL 1087/25)"",""siglaOrgao"":""PL108725"",""titulo"":""Suplente"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/539771""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2025-05-08T00:00"",""idOrgao"":539772,""nomeOrgao"":""Comissão Especial destinada a proferir parecer ao Projeto de Lei Complementar nº 234, de 2024, do Sr. Felipe Carreras, que \""dispõe sobre incentivos e benefícios para fomentar as atividades de caráter desportivo e dá outras providências\"""",""nomePublicacao"":""Comissão Especial da Lei de Incentivo ao Esporte (PLP 234/24)"",""siglaOrgao"":""PLP23424"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/539772""},{""codTitulo"":""101"",""dataFim"":null,""dataInicio"":""2025-06-30T00:00"",""idOrgao"":5971,""nomeOrgao"":""Conselho de Ética e Decoro Parlamentar da Câmara dos Deputados"",""nomePublicacao"":""Conselho de Ética e Decoro Parlamentar"",""siglaOrgao"":""COETICA"",""titulo"":""Titular"",""uriOrgao"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/5971""}]}",6163d285a5b457d667967233b25944c3827819ecf5841936dbfa016005f2351e,2023


In [66]:
%%sql
DROP TABLE IF EXISTS deputados_orgaos_camara;

CREATE TABLE deputados_orgaos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_orgaos
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.idOrgao') AS INTEGER) AS id_orgao,
    CAST(json_extract_string(e.value, '$.codTitulo') AS INTEGER) AS cod_titulo,
    CAST(json_extract_string(e.value, '$.dataInicio') AS DATETIME) AS data_inicio,
    CAST(json_extract_string(e.value, '$.dataFim') AS DATETIME) AS data_fim,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT id_deputado, id_orgao, cod_titulo, data_inicio, data_fim, year_snapshot
FROM exploded;

Running query in 'duck'

Count
1390


In [67]:
%%sql
SELECT * FROM deputados_orgaos_camara LIMIT 5;

Running query in 'duck'

id_deputado,id_orgao,cod_titulo,data_inicio,data_fim,year_snapshot
66385,5971,101,2025-06-30 00:00:00,None,2023
122974,539731,2,2025-06-10 00:00:00,None,2023
161440,539776,102,2025-05-21 00:00:00,None,2023
180214,539815,102,2025-09-12 00:00:00,None,2023
204406,539817,101,2025-07-07 00:00:00,None,2023


### Eventos e órgaos

In [68]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/eventos/orgaos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos/orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_eventos_orgaos;

Running query in 'duck'

n
707


In [69]:
%%sql
SELECT * FROM bronze_camara_eventos_orgaos LIMIT 5;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,eventos/orgaos,58552,https://dadosabertos.camara.leg.br/api/v2/eventos/58552/orgaos,"{""dados"":[{""apelido"":""PEC 015/15 - Fundeb"",""codTipoOrgao"":3,""id"":538357,""nome"":""Comissão Especial destinada a proferir parecer à Proposta de Emenda à Constituição nº 15-A, de 2015, da Srª Raquel Muniz e outros, que \""insere parágrafo único no art. 193; inciso IX, no art. 206 e art. 212-A, todos na Constituição Federal, de forma a tornar o Fundo de Manutenção e Desenvolvimento da Educação Básica e de Valorização dos Profissionais da Educação - Fundeb instrumento permanente de financiamento da educação básica pública, incluir o planejamento na ordem social e inserir novo princípio no rol daqueles com base nos quais a educação será ministrada, e revoga o art. 60 do Ato das Disposições Constitucionais Transitórias\"""",""nomePublicacao"":""Comissão Especial sobre o Fundeb (PEC 015/15)"",""nomeResumido"":""Fundeb"",""sigla"":""PEC01515"",""tipoOrgao"":""Comissão Especial"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/538357""}]}",8a346dde393754922bbb0b04fd92fa5301bdc742873dd5cc81d3f5777dbef338,2020
camara,eventos/orgaos,59113,https://dadosabertos.camara.leg.br/api/v2/eventos/59113/orgaos,"{""dados"":[{""apelido"":""PL 3515/15 - Superendividamento do Consumidor"",""codTipoOrgao"":3,""id"":538315,""nome"":""Comissão Especial destinada a proferir parecer ao Projeto de Lei nº 3515, de 2015, do Senado Federal, que \""altera a Lei nº 8.078, de 11 de setembro de 1990 (Código de Defesa do Consumidor), e o art. 96 da Lei nº 10.741, de 1º de outubro de 2003 (Estatuto do Idoso), para aperfeiçoar a disciplina do crédito ao consumidor e dispor sobre a prevenção e o tratamento do superendividamento\"", e apensados"",""nomePublicacao"":""Comissão Especial sobre Superendividamento do Consumidor (PL 3515/15)"",""nomeResumido"":""Superendividamento"",""sigla"":""PL351515"",""tipoOrgao"":""Comissão Especial"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/538315""}]}",d4a5744d49427b90e08289edcf0dafad93270e182c3a4e80321d2d73a97f7631,2020
camara,eventos/orgaos,59129,https://dadosabertos.camara.leg.br/api/v2/eventos/59129/orgaos,"{""dados"":[{""apelido"":""PL 0399/15 - Medicamentos Formulados com Cannabis"",""codTipoOrgao"":3,""id"":538408,""nome"":""Comissão Especial destinada a proferir parecer ao Projeto de Lei nº 399, de 2015, do Sr. Fábio Mitidieri, que \""altera o art. 2º da Lei nº 11.343, de 23 de agosto de 2006, para viabilizar a comercialização de medicamentos que contenham extratos, substratos ou partes da planta Cannabis sativa em sua formulação\"""",""nomePublicacao"":""Comissão Especial sobre Medicamentos Formulados com Cannabis (PL 0399/15)"",""nomeResumido"":""Remédios com Cannabis"",""sigla"":""PL039915"",""tipoOrgao"":""Comissão Especial"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/538408""}]}",ae5179feb98e5dd5dce001eb849c3c18283c6a2892622316c8662ce1eae4c07a,2020
camara,eventos/orgaos,59130,https://dadosabertos.camara.leg.br/api/v2/eventos/59130/orgaos,"{""dados"":[{""apelido"":""PL 0399/15 - Medicamentos Formulados com Cannabis"",""codTipoOrgao"":3,""id"":538408,""nome"":""Comissão Especial destinada a proferir parecer ao Projeto de Lei nº 399, de 2015, do Sr. Fábio Mitidieri, que \""altera o art. 2º da Lei nº 11.343, de 23 de agosto de 2006, para viabilizar a comercialização de medicamentos que contenham extratos, substratos ou partes da planta Cannabis sativa em sua formulação\"""",""nomePublicacao"":""Comissão Especial sobre Medicamentos Formulados com Cannabis (PL 0399/15)"",""nomeResumido"":""Remédios com Cannabis"",""sigla"":""PL039915"",""tipoOrgao"":""Comissão Especial"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/orgaos/538408""}]}",ae5179feb98e5dd5dce001eb849c3c18283c6a2892622316c8662ce1eae4c07a,2020
camara,eventos/orgaos,59186,https://dadosabertos.camara.leg.br/api/v2/eventos/59186/orgaos,"{""dados"":[{""apelido"":""Vi

In [70]:
%%sql
DROP TABLE IF EXISTS eventos_orgaos_camara;

CREATE TABLE eventos_orgaos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_evento
  FROM bronze_camara_eventos_orgaos
),
exploded AS (
  SELECT
    CAST(b.id_evento AS BIGINT) AS id_evento,
    CAST(json_extract_string(e.value, '$.id') AS INTEGER) AS id_orgao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT id_evento, id_orgao, year_snapshot
FROM exploded;

Running query in 'duck'

Count
707


In [71]:
%%sql
SELECT * FROM eventos_orgaos_camara LIMIT 5;

Running query in 'duck'

id_evento,id_orgao,year_snapshot
58552,538357,2020
59113,538315,2020
59129,538408,2020
59130,538408,2020
59186,538364,2020


### Eventos e pautas

In [72]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos_pauta AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/eventos/pauta/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos/pauta';

SELECT COUNT(*) AS n FROM bronze_camara_eventos_pauta;

Running query in 'duck'

n
707


In [73]:
%%sql
SELECT * FROM bronze_camara_eventos_pauta LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,eventos/pauta,58552,https://dadosabertos.camara.leg.br/api/v2/eventos/58552/pauta,"{""dados"":[{""codRegime"":106,""ordem"":1,""proposicaoRelacionada_"":null,""proposicao_"":{""ano"":2015,""codTipo"":136,""ementa"":""Insere parágrafo único no art. 193; inciso IX, no art. 206 e art. 212-A, todos na Constituição Federal, de forma a tornar o Fundo de Manutenção e Desenvolvimento da Educação Básica e de Valorização dos Profissionais da Educação - Fundeb instrumento permanente de financiamento da educação básica pública, incluir o planejamento na ordem social e inserir novo princípio no rol daqueles com base nos quais a educação será ministrada, e revoga o art. 60 do Ato das Disposições Constitucionais Transitórias. NOVA EMENTA: Altera a Constituição Federal para estabelecer critérios de distribuição da cota municipal do Imposto Sobre Operações Relativas à Circulação de Mercadorias e sobre Prestações de Serviços de Transporte Interestadual e Intermunicipal e de Comunicação (ICMS), para disciplinar a disponibilização de dados contábeis pelos entes federados, para tratar do planejamento na ordem social e para dispor sobre o Fundo de Manutenção e Desenvolvimento da Educação Básica e de Valorização dos Profissionais da Educação (Fundeb); altera o Ato das Disposições Constitucionais Transitórias; e dá outras providências"",""id"":1198512,""numero"":15,""siglaTipo"":""PEC"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/proposicoes/1198512""},""regime"":""Especial (Art. 202 c/c 191, I, RICD)"",""relator"":{""email"":""dep.professoradorinhaseabrarezende@camara.leg.br"",""id"":160639,""idLegislatura"":55,""nome"":""Professora Dorinha Seabra Rezende"",""siglaPartido"":""DEM"",""siglaUf"":""TO"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/160639"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36769"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/160639.jpg""},""situacaoItem"":""Vista conjunta aos Deputados Adriana Ventura, Alice Portugal, Angela Amin, Átila Lira, Benes Leocádio, Chris Tonietto, Coronel Armando, Daniela do Waguinho, Danilo Cabral, Dr. Jaziel, Dra. Soraya Manato, Edmilson Rodrigues, Eduardo Braide, Emanuel Pinheiro Neto, Felipe Rigoni, Fernando Rodolfo, Gastão Vieira, General Peternelli, Haroldo Cathedral, Helio Lopes, Idilvan Alencar, Igor Timo, Jéssica Sales, João H. Campos, Juarez Costa, Leda Sadala, Lídice da Mata, Luisa Canziani, Luizão Goulart, Marcelo Calero, Maria Rosas, Mariana Carvalho, Marreca Filho, Natália Bonavides, Osires Damaso, Otoni de Paula, Paula Belmonte, Pedro Cunha Lima, Pedro Uczai, Professor Alcides, Professor Israel Batista, Professora Dayane Pimentel, Professora Marcivania, Professora Rosa Neide, Raul Henry, Rejane Dias, Rogério Correia, Rosana Valle, Rose Modesto, Sâmia Bomfim, Sidney Leite, Tabata Amaral, Tereza Nelma, Tiago Dimas, Túlio Gadêlha, Weliton Prado, Zé Vitor e Zeca Dirceu."",""textoParecer"":null,""titulo"":""PEC 15/2015"",""topico"":""Disposições Especiais"",""uriVotacao"":null}]}",bd68b5c89f15e81094955070a90d7423aab92ffad03062feaa9da313cd23a23e,2020


In [74]:
import pyarrow.parquet as pq
import json
import pandas as pd
import tempfile
from glob import glob

with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-smoke/camara/eventos/pauta/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    df.head()
    exploded_rows = []

    for index, row in df.iterrows():
        payload = json.loads(row['payload_json'])
        for item in payload['dados']:
            exploded_rows.append({
                'id_evento': row['id'],
                'cod_regime': item['codRegime'],
                'ordem': item['ordem'],
                'id_proposicao': item['proposicao_']['id'],
                'id_relator': item['relator']['id'] if item['relator'] else None,
                'year_snapshot': row['year'],
            })
    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name)

    conn.execute(f"""
        DROP TABLE IF EXISTS eventos_pauta_camara;
        CREATE TABLE eventos_pauta_camara AS
            SELECT
                id_evento::BIGINT AS id_evento,
                cod_regime::TEXT AS cod_regime,
                ordem::INTEGER AS ordem,
                id_proposicao::BIGINT AS id_proposicao,
                id_relator::BIGINT AS id_relator,
                year_snapshot::INTEGER AS year_snapshot
            FROM read_csv_auto('{tf.name}', HEADER=TRUE);
    """)

### Legislaturas e líderes

In [75]:
import pyarrow.parquet as pq
import json
import pandas as pd
import tempfile
from glob import glob

with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-smoke/camara/legislaturas/lideres/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    df.head()
    exploded_rows = []

    for index, row in df.iterrows():
        payload = json.loads(row['payload_json'])
        for item in payload['dados']:
            exploded_rows.append({
                'id_legislatura': row['id'],
                'nome_bancada': item['bancada']['nome'],
                'tipo_bancada': item['bancada']['tipo'],
                'uri_bancada': item['bancada']['uri'],
                'data_inicio': item['dataInicio'],
                'data_fim': item['dataFim'],
                'id_deputado': item['parlamentar']['id'],
                'titulo': item['titulo'],
                'year_snapshot': row['year'],
            })
    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name)

    conn.execute(f"""
        DROP TABLE IF EXISTS legislaturas_lideres_camara;
        CREATE TABLE legislaturas_lideres_camara AS
            SELECT
                id_legislatura::BIGINT AS id_legislatura,
                nome_bancada::TEXT AS nome_bancada,
                tipo_bancada::TEXT as tipo_bancada,
                uri_bancada::TEXT as uri_bancada,
                data_inicio::DATETIME as data_inicio,
                data_fim::DATETIME as data_fim,
                id_deputado::BIGINT AS id_deputado,
                titulo::TEXT AS titulo,
                year_snapshot::INTEGER AS year_snapshot
            FROM read_csv_auto('{tf.name}', HEADER=TRUE);
    """)

In [76]:
%%sql
SELECT * from legislaturas_lideres_camara LIMIT 5;

Running query in 'duck'

id_legislatura,nome_bancada,tipo_bancada,uri_bancada,data_inicio,data_fim,id_deputado,titulo,year_snapshot
52,Governo na CD,Governo na CD,None,2003-02-01 00:00:00,2004-01-23 00:00:00,73428,Líder,2003
52,PDT,Partido Político,https://dadosabertos.camara.leg.br/api/v2/partidos/36786,2003-02-01 00:00:00,2004-02-12 00:00:00,74225,Líder,2003
52,PSB,Partido Político,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,2003-02-01 00:00:00,2004-01-28 00:00:00,74416,Líder,2003
52,PL*,Partido Político,https://dadosabertos.camara.leg.br/api/v2/partidos/36795,2003-02-01 00:00:00,2004-02-18 00:00:00,73565,Líder,2003
52,PSDB,Partido Político,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,2003-02-01 00:00:00,2004-02-17 00:00:00,74570,Líder,2003


### Legislaturas e mesa

In [77]:

%%sql
CREATE OR REPLACE VIEW bronze_camara_legislaturas_mesa AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/legislaturas/mesa/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'legislaturas/mesa';

SELECT COUNT(*) AS n FROM bronze_camara_legislaturas_mesa;

Running query in 'duck'

n
57


In [78]:
%%sql
SELECT * FROM bronze_camara_legislaturas_mesa LIMIT 1;

Running query in 'duck'

+--------+-------------------+----+-------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [79]:

%%sql
DROP TABLE IF EXISTS legislaturas_mesa_camara;
CREATE TABLE legislaturas_mesa_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, 
    year AS year_snapshot,
    id AS id_legislatura,
    FROM bronze_camara_legislaturas_mesa
),
exploded as (
    SELECT
        CAST(base.id_legislatura AS BIGINT) AS id_legislatura,
        -- id
        CAST(jget1(elem.value, '$.id') AS BIGINT) AS id_deputado,
        -- codTitulo
        jget1(elem.value, '$.codTitulo') AS cod_titulo,
        -- dataInicio
        CAST(jget1(elem.value, '$.dataInicio') AS DATE) AS data_inicio,
        -- dataFim
        CAST(jget1(elem.value, '$.dataFim') AS DATE) AS data_fim,
        year_snapshot,
    FROM base
    CROSS JOIN json_each(json_extract(j, '$.dados')) AS elem
)
SELECT
    id_legislatura,
    id_deputado,
    cod_titulo,
    data_inicio,
    data_fim,
    year_snapshot
FROM exploded
WHERE id_legislatura IS NOT NULL AND id_deputado IS NOT NULL
ORDER BY id_legislatura, cod_titulo;




Running query in 'duck'

Count
144


### Partidos e líderes

In [80]:
%%sql
CREATE OR REPLACE VIEW bronze_partidos_lideres AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/partidos/lideres/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos/lideres';

SELECT COUNT(*) AS n FROM bronze_partidos_lideres;


Running query in 'duck'

n
20


In [81]:
%%sql
SELECT * FROM bronze_partidos_lideres;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,partidos/lideres,36835,https://dadosabertos.camara.leg.br/api/v2/partidos/36835/lideres,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,partidos/lideres,36839,https://dadosabertos.camara.leg.br/api/v2/partidos/36839/lideres,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,partidos/lideres,36844,https://dadosabertos.camara.leg.br/api/v2/partidos/36844/lideres,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,partidos/lideres,36886,https://dadosabertos.camara.leg.br/api/v2/partidos/36886/lideres,"{""dados"":[]}",e3f60f959291f48551b8f9a819aa280d6c26cb36048a77675fc4e4107d2f7b49,2020
camara,partidos/lideres,36834,https://dadosabertos.camara.leg.br/api/v2/partidos/36834/lideres,"{""dados"":[{""codTitulo"":1001,""dataFim"":null,""dataInicio"":""2023-02-01"",""email"":""dep.antoniobrito@camara.leg.br"",""id"":160553,""idLegislatura"":57,""nome"":""Antonio Brito"",""siglaPartido"":""PSD"",""siglaUf"":""BA"",""titulo"":""Líder"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/160553"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36834"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/160553.jpgmaior.jpg""}]}",93f726030e3401a7f39b6a3dc4dbfa3fcf436307c83f47d295f0d3e0ee2b3729,2023
camara,partidos/lideres,36896,https://dadosabertos.camara.leg.br/api/v2/partidos/36896/lideres,"{""dados"":[{""codTitulo"":1,""dataFim"":null,""dataInicio"":""2023-02-01"",""email"":""dep.renataabreu@camara.leg.br"",""id"":178989,""idLegislatura"":57,""nome"":""Renata Abreu"",""siglaPartido"":""PODE"",""siglaUf"":""SP"",""titulo"":""Presidente"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/178989"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36896"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/178989.jpgmaior.jpg""},{""codTitulo"":1001,""dataFim"":null,""dataInicio"":""2025-02-03"",""email"":""dep.rodrigogambale@camara.leg.br"",""id"":220641,""idLegislatura"":57,""nome"":""Rodrigo Gambale"",""siglaPartido"":""PODE"",""siglaUf"":""SP"",""titulo"":""Líder"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/220641"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36896"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/220641.jpgmaior.jpg""}]}",7da835807a31c8072276a79413413c6ec19132ab15959dcc36225a09855f6775,2023
camara,partidos/lideres,36898,https://dadosabertos.camara.leg.br/api/v2/partidos/36898/lideres,"{""dados"":[{""codTitulo"":1,""dataFim"":null,""dataInicio"":""2023-02-01"",""email"":""dep.luistibe@camara.leg.br"",""id"":160510,""idLegislatura"":57,""nome"":""Luis Tibé"",""siglaPartido"":""AVANTE"",""siglaUf"":""MG"",""titulo"":""Presidente"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/160510"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36898"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/160510.jpgmaior.jpg""},{""codTitulo"":1001,""dataFim"":null,""dataInicio"":""2025-05-08"",""email"":""dep.netocarletto@camara.leg.br"",""id"":220703,""idLegislatura"":57,""nome"":""Neto Carletto"",""siglaPartido"":""AVANTE"",""siglaUf"":""BA"",""titulo"":""Líder"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/220703"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36898"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/220703.jpgmaior.jpg""}]}",8dca202d767499b190864d041b342e0317d559e42b8b57f997b620299f57aa89,2023
camara,partidos/lideres,36899,https://dadosabertos.camara.leg.br/api/v2/partidos/36899/lideres,"{""dados"":[{""codTitulo"":1,""dataFim"":null,""dataInicio"":""2023-02-01"",""email"":""dep.baleiarossi@camara.leg.br"",""id"":178975,""idLegislatura"":57,""nome"":""Baleia Rossi"",""siglaPartido"":""MDB

In [82]:
%%sql
DROP TABLE IF EXISTS partidos_lideres_camara;

CREATE TABLE partidos_lideres_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_partido,
  FROM bronze_partidos_lideres
),
exploded AS (
  SELECT
    CAST(b.id_partido AS BIGINT) AS id_partido,
      -- codTitulo
    json_extract_string(e.value, '$.codTitulo') AS cod_titulo,
    -- dataInicio
    CAST(json_extract_string(e.value, '$.dataInicio') AS DATE) AS data_inicio,
    -- dataFim
    CAST(json_extract_string(e.value, '$.dataFim') AS DATE) AS data_fim,
    -- id (deputado)
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_deputado,
    -- year
    b.year_snapshot,
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_partido,
    cod_titulo,
    data_inicio,
    data_fim,
    id_deputado,
    year_snapshot,
FROM exploded;
DROP VIEW IF EXISTS bronze_partidos_lideres;

Running query in 'duck'

Success


In [83]:
%%sql
SELECT * FROM partidos_lideres_camara LIMIT 5;

Running query in 'duck'

id_partido,cod_titulo,data_inicio,data_fim,id_deputado,year_snapshot
36834,1001,2023-02-01,None,160553,2023
36896,1001,2025-02-03,None,220641,2023
36898,1001,2025-05-08,None,220703,2023
36899,1001,2023-02-01,None,204436,2023
37903,1001,2023-09-12,None,204450,2023


### Partidos e membros

In [84]:
%%sql
CREATE OR REPLACE VIEW bronze_partidos_membros AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-smoke/camara/partidos/membros/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos/membros';

SELECT COUNT(*) AS n FROM bronze_partidos_membros;

Running query in 'duck'

n
20


In [85]:
%%sql
SELECT * FROM bronze_partidos_membros LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
camara,partidos/membros,36779,https://dadosabertos.camara.leg.br/api/v2/partidos/36779/membros,"{""dados"":[{""email"":""dep.renildocalheiros@camara.leg.br"",""id"":73801,""idLegislatura"":57,""nome"":""Renildo Calheiros"",""siglaPartido"":""PCdoB"",""siglaUf"":""PE"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/73801"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/73801.jpg""},{""email"":""dep.aliceportugal@camara.leg.br"",""id"":74057,""idLegislatura"":57,""nome"":""Alice Portugal"",""siglaPartido"":""PCdoB"",""siglaUf"":""BA"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/74057"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/74057.jpg""},{""email"":""dep.danielalmeida@camara.leg.br"",""id"":74060,""idLegislatura"":57,""nome"":""Daniel Almeida"",""siglaPartido"":""PCdoB"",""siglaUf"":""BA"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/74060"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/74060.jpg""},{""email"":""dep.jandirafeghali@camara.leg.br"",""id"":74848,""idLegislatura"":57,""nome"":""Jandira Feghali"",""siglaPartido"":""PCdoB"",""siglaUf"":""RJ"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/74848"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/74848.jpg""},{""email"":""dep.marciojerry@camara.leg.br"",""id"":81055,""idLegislatura"":57,""nome"":""Márcio Jerry"",""siglaPartido"":""PCdoB"",""siglaUf"":""MA"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/81055"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/81055.jpg""},{""email"":""dep.professoramarcivania@camara.leg.br"",""id"":160641,""idLegislatura"":57,""nome"":""Professora Marcivania"",""siglaPartido"":""PCdoB"",""siglaUf"":""AP"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/160641"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/160641.jpg""},{""email"":""dep.orlandosilva@camara.leg.br"",""id"":178987,""idLegislatura"":57,""nome"":""Orlando Silva"",""siglaPartido"":""PCdoB"",""siglaUf"":""SP"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/178987"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/178987.jpg""},{""email"":""dep.daianasantos@camara.leg.br"",""id"":220555,""idLegislatura"":57,""nome"":""Daiana Santos"",""siglaPartido"":""PCdoB"",""siglaUf"":""RS"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/220555"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/220555.jpg""},{""email"":""dep.enfermeirarejane@camara.leg.br"",""id"":230765,""idLegislatura"":57,""nome"":""Enfermeira Rejane"",""siglaPartido"":""PCdoB"",""siglaUf"":""RJ"",""uri"":""https://dadosabertos.camara.leg.br/api/v2/deputados/230765"",""uriPartido"":""https://dadosabertos.camara.leg.br/api/v2/partidos/36779"",""urlFoto"":""https://www.camara.leg.br/internet/deputado/bandep/230765.jpg""}]}",1bedef9aea98e82f42ffe0e51a7c851e82c44c891c0fec24fc27f2ba762b9745,2020


In [86]:
%%sql
DROP TABLE IF EXISTS partidos_membros_camara;

CREATE TABLE partidos_membros_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_partido,
  FROM bronze_partidos_membros
),
exploded AS (
  SELECT
    CAST(b.id_partido AS BIGINT) AS id_partido,
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_deputado,
    CAST(json_extract_string(e.value, '$.id_legislatura') AS INTEGER) AS id_legislatura,
    b.year_snapshot,
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_partido,
    id_deputado,
    id_legislatura,
    year_snapshot,
FROM exploded;
DROP VIEW IF EXISTS bronze_partidos_membros;

Running query in 'duck'

Success


In [87]:
%%sql
SELECT * FROM partidos_membros_camara LIMIT 10;

Running query in 'duck'

id_partido,id_deputado,id_legislatura,year_snapshot
36779,230765,None,2020
36786,229082,None,2020
36832,220686,None,2020
36834,228042,None,2020
36835,220683,None,2020
36839,233592,None,2020
36844,231911,None,2020
36851,220697,None,2020
36886,220637,None,2020
36896,233598,None,2020
